In [1]:
import os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from keras.models import Sequential
from keras.layers import TimeDistributed, LSTM, Dense, RepeatVector
from numpy import array, array_equal, argmax
import tensorflow as tf
import attention_decoder

"""
Sample keras model
"""
def create_model_1():
    Note = Input(shape=(sequence_length, n_vocab), dtype = 'float32')
    y = LSTM(400, input_shape = (sequence_length, n_vocab), return_sequences = True ) (Note)
    y = Dropout(0.4) (y)
    y = LSTM(400) (y)
    y = Dropout(0.4) (y)
    # Two weights, two bias
    # If coordinates of input is (X1, X2), and this layer is (Y1, Y2, Y3, Y4)
    # Result would be 
    y1 = Lambda(lambda x: x*2)( Dense(2, activation = 'tanh') (y) )
    y2 = Dense(2, activation = 'linear') (y)
    Coordinates = Input(shape= (2, ), dtype = 'float32')
    print (keras.backend.shape(y1))
    print (keras.backend.shape(Coordinates))
    c1 = Multiply()([y1, Coordinates])
    c = Add()([y2, c1])
    m = Model(inputs = [Note, Coordinates], outputs = c)

    print (m.summary())
    m.compile(loss='mean_squared_error', optimizer='adam')

    return m

def create_sample_attention_model () :
    model = Sequential()
    model.add(LSTM(150, input_shape=(n_timesteps_in, n_features), return_sequences=True))
    model.add(attention_decoder.AttentionDecoder(150, n_features))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

Using TensorFlow backend.


Seems like for this problem, I need to use Tensorflow

In [2]:
output_ta = tf.TensorArray(size=784, dtype=tf.float32)  # store trained/sampled pixel
def loop_fn(time, cell_output, cell_state, loop_state):
    emit_output = cell_output  # == None for time == 0
    if cell_output is None:
        # time=0, everything here will be used for initialization only
        next_cell_state = cell_init_state
        next_pixel = cell_init_pixel
        next_loop_state = output_ta
    else:  
        # pass the last state to the next
        next_cell_state = cell_state
        next_pixel = tf.cond(is_training,
                         lambda: inputs_ta.read(time - 1),
                         lambda: tf.contrib.distributions.Bernoulli(
                             probs=tf.nn.sigmoid(tf.layers.dense(cell_output, 1, 
                                 name='output_to_p', activation=tf.nn.sigmoid,
                                 reuse=True)),
                             dtype=tf.float32).sample())
        next_loop_state = loop_state.write(time - 1, next_pixel)
    elements_finished = (time >= 784)
    return (elements_finished, next_pixel, next_cell_state,
            emit_output, next_loop_state)

In [6]:
from tensorflow.python.ops.rnn import _transpose_batch_time

def sampling_rnn(self, cell, initial_state, input_, seq_lengths):

    # raw_rnn expects time major inputs as TensorArrays
    max_time = ...  # this is the max time step per batch
    inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_time, clear_after_read=False)
    inputs_ta = inputs_ta.unstack(_transpose_batch_time(input_))  # model_input is the input placeholder
    input_dim = input_.get_shape()[-1].value  # the dimensionality of the input to each time step
    output_dim = ...  # the dimensionality of the model's output at each time step
    
    def loop_fn(time, cell_output, cell_state, loop_state):
        """
        Loop function that allows to control input to the rnn cell and manipulate cell outputs.
        :param time: current time step
        :param cell_output: output from previous time step or None if time == 0
        :param cell_state: cell state from previous time step
        :param loop_state: custom loop state to share information between different iterations of this loop fn
        :return: tuple consisting of
          elements_finished: tensor of size [bach_size] which is True for sequences that have reached their end,
            needed because of variable sequence size
          next_input: input to next time step
          next_cell_state: cell state forwarded to next time step
          emit_output: The first return argument of raw_rnn. This is not necessarily the output of the RNN cell,
            but could e.g. be the output of a dense layer attached to the rnn layer.
          next_loop_state: loop state forwarded to the next time step
        """
        if cell_output is None:
            # time == 0, used for initialization before first call to cell
            next_cell_state = initial_state
            # the emit_output in this case tells TF how future emits look
            emit_output = tf.zeros([output_dim])
        else:
            # t > 0, called right after call to cell, i.e. cell_output is the output from time t-1.
            # here you can do whatever ou want with cell_output before assigning it to emit_output.
            # In this case, we don't do anything
            next_cell_state = cell_state
            emit_output = cell_output  

        # check which elements are finished
        elements_finished = (time >= seq_lengths)
        finished = tf.reduce_all(elements_finished)

        # assemble cell input for upcoming time step
        current_output = emit_output if cell_output is not None else None
        input_original = inputs_ta.read(time)  # tensor of shape (None, input_dim)

        if current_output is None:
            # this is the initial step, i.e. there is no output from a previous time step, what we feed here
            # can highly depend on the data. In this case we just assign the actual input in the first time step.
            next_in = input_original
        else:
            # time > 0, so just use previous output as next input
            # here you could do fancier things, whatever you want to do before passing the data into the rnn cell
            # if here you were to pass input_original than you would get the normal behaviour of dynamic_rnn
            next_in = current_output

        next_input = tf.cond(finished,
                             lambda: tf.zeros([self.batch_size, input_dim], dtype=tf.float32),  # copy through zeros
                             lambda: next_in)  # if not finished, feed the previous output as next input

        # set shape manually, otherwise it is not defined for the last dimensions
        next_input.set_shape([None, input_dim])

        # loop state not used in this example
        next_loop_state = None
        return (elements_finished, next_input, next_cell_state, emit_output, next_loop_state)

    outputs_ta, last_state, _ = tf.nn.raw_rnn(cell, loop_fn)
    outputs = _transpose_batch_time(outputs_ta.stack())
    final_state = last_state

    return outputs, final_state

In [5]:
import cv2
import os
import pylab as pl
%matplotlib inline

### Image size 

(288, 432, 3)

In [151]:
cap = cv2.VideoCapture(os.path.join('..', 'target', '0', '0.mp4'))

In [155]:
def crop_and_resize (frame):
    
    # cv2.resize(frame, (10, 10), interpolation = cv2.INTER_AREA)
#     return frame[36:250, 114:328, :]
    return cv2.resize(frame[36:250, 114:328, :], (15, 15), interpolation = cv2.INTER_LINEAR)

In [156]:
data = []

counter = 0
while (cap.isOpened()):
    ret, frame = cap.read()

    if not ret:
        break
        
    data.append(crop_and_resize(frame))
    
    counter += 1

In [157]:
pl.imshow(data[18])

IndexError: list index out of range

### Get instruction signal

Parse the video file into instruction sequence

In [54]:
instruction_dict = {'north': 0, 'east': 1, 'south': 2, 'west': 3}

In [55]:
reverse_instruction_dict = dict((v,k) for (k,v) in instruction_dict.items())

In [226]:
def decode ( sequence ):
    return ','.join([reverse_instruction_dict[s] for s in sequence])

In [237]:
def get_instruction_btw_frame( prev_frame, cur_frame ):
    diff = abs(prev_frame.astype(np.int32) - cur_frame.astype(np.int32))
    vals = np.argwhere(diff >= 20)
    
    points = set()
    for val in vals:
        points.add(tuple(val[:2]))
    
    # if prev_frame[points[0]] ~ (255,255,255), points[0] is target, points[1] is start
    # otherwise reverse
    
    points = list(points)
    
    if np.sum(abs(prev_frame[points[0]].astype(np.int32) - np.array([255, 255, 255]))) > 20:
        source = np.array(points[0])
        target = np.array(points[1])
    else:
        source = np.array(points[1])
        target = np.array(points[0])

        
    if np.array_equal(target - source , [0, 1]):
        return 1
    if np.array_equal(target - source , [0, -1]):
        return 3
    if np.array_equal(target - source , [1, 0]):
        return 2
    if np.array_equal(target - source , [-1, 0]):
        return 0
        

def get_instructions ( video_path, debug = True ):
    cap = cv2.VideoCapture(video_path)
    
    prev_frame = None
    instructions = []

    counter = 0
    while (cap.isOpened()):
        ret, frame = cap.read()

        if not ret:
            break
            
        c_frame = crop_and_resize(frame)
        
        # Get the instruction between prev_frame and frame
        if prev_frame is not None:
            instruction = get_instruction_btw_frame (prev_frame, c_frame)
            instructions.append(instruction)

        counter += 1

        prev_frame = c_frame
        
    #pl.imshow(prev_frame)
    
    if debug :
        print ('There are %d frames ' % counter)
        print ('The list of instructions are %s ' % decode( instructions ))
    
    return instructions

In [238]:
get_instructions(os.path.join('..', 'target', '0', '0.mp4'))

There are 19 frames 
The list of instructions are south,south,south,south,south,south,west,west,south,south,west,west,west,west,west,west,south,west 


[2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3]

### Baseline model

LSTM model with attention with the image being additional input to encoded vector.

The simplest model would have image flattened out into an 100-cell vectors. Colors are coded.